# Dataset continue test

We use a sorted enwiki_100k and interrupt midway, and continue from checkpoint
We validate that the dataset curve follows as expected.

In [2]:
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-dataset-continue-test"

# We limit to 1 for this test
GPU_DEVICES="1"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: 1
NOTEBOOK_DIR: /workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /workspace/picocreator/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /workspace/picocreator/RWKV-infctx-trainer


In [3]:
# Lets initialized the L6-D512 model with the init_model.py code
!cd "{TRAINER_DIR}" && python3 init_model.py \
    --n_layer 6 --n_embd 512 \
    --vocab_size world \
    --skip-if-exists --safe-init \
    ../model/L6-D512-world-init.pth

[2024-02-10 19:49:19,920] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.1+cu121'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-world-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


In [6]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-sorted.yaml"

Setting num_proc from 160 to 6 for the train split as it only contains 6 shards.
Generating train split: 100%|█| 1000000/1000000 [00:03<00:00, 259169.30 examples
Map (num_proc=160): 100%|███| 1000000/1000000 [00:17<00:00, 56840.38 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:05<00:00, 169992.69 examples/
Map (num_proc=160): 100%|█████| 467553/467553 [00:05<00:00, 77935.36 examples/s]
Saving the dataset (7/7 shards): 100%|█| 467553/467553 [00:10<00:00, 46628.18 ex
Saving the dataset (1/1 shards): 100%|█| 4723/4723 [00:00<00:00, 38762.97 exampl


# Initial Fit

In [26]:
!cd "{TRAINER_DIR}" && \
    mkdir -p "{PROJECT_DIR}/checkpoint/v5-enwiki-100k-sorted/" && \
    rm -rf "{PROJECT_DIR}/checkpoint/v5-enwiki-100k-sorted/*" && pwd

/workspace/picocreator/RWKV-infctx-trainer/RWKV-v5


In [27]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-sorted.yaml" \
        --model.load_model="../model/L6-D512-world-init.pth" \
        --trainer.max_steps=251 \
        --trainer.callbacks.init_args.every_n_train_steps=50 \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v5-enwiki-100k-sorted/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - PART 1 (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=8 \
        --data.skip_datapath_setup=True \
        --trainer.devices="{GPU_DEVICES}"

[2024-02-11 01:58:48,525] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.1+cu121'
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-sorted.yaml', '--model.load_model=../model/L6-D512-world-init.pth', '--trainer.max_steps=251', '--trainer.callbacks.init_args.every_n_train_steps=50', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-enwiki-100k-sorted/', '--trainer.logger.init_args.name=infctx-v5-dataset-continue-test - PART 1 (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_1', '--train

In [26]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-sorted.yaml" \
        --model.load_model="../model/L6-D512-world-init.pth" \
        --trainer.callbacks.init_args.every_n_train_steps=5000 \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v5-enwiki-100k-sorted/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - PART 2 (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=8 \
        --trainer.devices="{GPU_DEVICES}" \
        --data.skip_datapath_setup=True \
        --ckpt_path="../checkpoint/v5-enwiki-100k-sorted/epoch=0-step=250.ckpt"

[2024-02-11 05:22:01,976] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.1+cu121'
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/workspace/picocreator/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_100k-world-sorted.yaml', '--model.load_model=../model/L6-D512-world-init.pth', '--trainer.max_steps=500', '--trainer.callbacks.init_args.every_n_train_steps=5000', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-enwiki-100k-sorted/', '--trainer.logger.init_args.name=infctx-v5-dataset-continue-test - PART 2 (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_1', '--tra